In [2]:
import requests
import pandas as pd 
import json
import datetime 
import csv
import praw
import json
from datetime import datetime
import glob
import os
from psaw import PushshiftAPI
api = PushshiftAPI()

In [ ]:
#got it from reddit
CLIENT_ID = "" 
SECRET_KEY = ""

In [ ]:
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY) #getting authorizations
auth

In [ ]:
#putting password in txt file instead of writing it down
f = open("password_reddit.txt","r")
lines=f.readlines()
password=lines[0]
f.close

In [ ]:
#reddit API needs this in this format
data = {
    'grant_type': 'password',
    'username': 'randomlymixed',
    'password': password
}
user_agent = "MyAPI 1.0"
headers = {'User-Agent': "MyAPI/0.0.1"}

In [ ]:
#reddit API 
res = requests.post('https://www.reddit.com/api/v1/access_token', 
                    auth=auth, data= data, headers=headers)

In [ ]:
#getting the token to access reddit api
TOKEN = res.json()['access_token']

In [ ]:
headers['Authorization'] = f'bearer {TOKEN}'
headers

In [1]:
#testing if I have access with my credentials 
requests.get('https://oauth.reddit.com/api/v1/me', headers = headers).json()

NameError: name 'requests' is not defined

In [ ]:
#this is for praw, another reddit scraper. I did not use it for this project but this is how I coded it 
reddit = praw.Reddit(
    client_id = CLIENT_ID,
    client_secret = SECRET_KEY,
    user_agent = user_agent
)

In [2]:
def get_reddit_data(after, before, term, sub, limit): #pushshift is the scraper I used, more info at https://github.com/pushshift/api
    ''' Purpose of the function is to get all reddit comments and additional with certain other informationkey terms 
    within the period of after and before days. All the reddit information is stored in a dictionary.
    after: is a string in the format of "#d". Ex: 720d
    before: is a string in the format of "#d". Ex: 500d
    term: key words that we will search in. String format. if spaces adds dashes instead. Ex: "Covid-Vaccine"
    sub: is a string in the format of "sub" Short for subreddit. Ex: "AskReddit"
    limit: how many comments should it pull. Max limit is 100. in a str format "#". Ex: "100"
    '''
    url = 'https://api.pushshift.io/reddit/search/comment/?q='+ str(term) + '&after=' +str(after)+ '&before='+str(before)+'&subreddit=' +str(sub) +'&size='+str(limit)
    print(url) #gives us the url so we can go to the site ourselves to see what is on it 
    res = requests.get(url) #running to get what we need 
    data = json.loads(res.text, strict = False) #puts it in a dictionary for us 
    return data['data'] #returning the important thing of the data

SyntaxError: invalid syntax (<ipython-input-2-dd65bdf98a28>, line 2)

In [ ]:
df_comments = pd.DataFrame() #initalizing a dataframe outside so I can have a copy of it
def comment_reddit(after, before, term, subreddit, limit):
    '''
    Purpose is to store all the comments in a dataframe that we scraped from the other function. Return it in a pd.DataFrame()
    after: is a string in the format of "#d". Ex: 720d
    before: is a string in the format of "#d". Ex: 500d
    after: is a string in the format of "#d". Ex: 720d
    before: is a string in the format of "#d". Ex: 500d
    term: key words that we will search in. String format. if spaces adds dashes instead. Ex: "Covid-Vaccine"
    sub: is a string in the format of "sub" Short for subreddit. Ex: "AskReddit"
    limit: how many comments should it pull. Max limit is 100. in a str format "#". Ex: "100"
    '''
    data = get_reddit_data(str(after), str(before), str(term), str(subreddit), str(limit)) #using other function 
    for submission in data: 
        df_comments = df_comments.append({'subreddit': submission['subreddit'],
                    'score': submission['score'],
                    'comments': submission['body'],
                    'time_created': datetime.utcfromtimestamp(submission['created_utc']).strftime('%Y-%m-%d %H:%M:%S')
                   }, ignore_index = True) #creates our df 
    #name_csv = str(input("Name of CSV: "))
    #df.to_csv(name_csv+".csv", index=False)
    return(df_comments)

In [ ]:
# ["vaccine-hate","anti-vaccine", "COVID-vaccine"] list that was used 
#the purpose was to create all the dfs we created into csv to store 
def data_collection_reddit(list_name, subreddit):
    '''
    The purpose was to create all the dfs we created into csv to store. Based on some requirements
    list_name: a list of key terms that should be searched on. List should be strings. Ex: ["vaccine-hate","anti-vaccine", "COVID-vaccine"]
    subreddit: what subreddit we want to search the terms on. In str format. Ex: "Republican"
    '''
    counter = 1 #to name the file easier 
    for i in list_name: 
        for j in range(720, 540, -20): #in increments of 20 days 
            try:
                df = comment_reddit(str(j)+"d", "540d", i, str(subreddit), '400')
                if df.empty:
                    continue
                else:
                    df.to_csv("Data/03_11_2020-09_07_2020, Subreddit-" + str(subreddit) + str(counter) +".csv", index=False)
                    counter += 1
            except:
                continue
    for i in list_name:
        for j in range(540, 360, -20):
            try:
                df = comment_reddit(str(j)+"d", "360d", i, str(subreddit), '400')
                if df.empty:
                    continue
                else:
                    df.to_csv("Data/09_11_2020-03_06_2021, Subreddit-" + str(subreddit) + str(counter) +".csv", index=False)
                    counter += 1
            except:
                continue
    
    for i in list_name:
        for j in range(360, 180, -15):
            try:
                df = comment_reddit(str(j)+"d", "180d", i, str(subreddit), '400')
                if df.empty:
                    continue
                else:
                    df.to_csv("Data/03_06_2021-09_02_2021, Subreddit-" + str(subreddit) + str(counter) +".csv", index=False)
                    counter += 1
            except:
                continue
    for i in list_name:
        for j in range(180, 0, -9):
            try:
                df = comment_reddit(str(j)+"d", "0d", i, str(subreddit), '400')
                if df.empty:
                    continue
                else:
                    df.to_csv("Data/09_02_2021-03_01_2022, Subreddit-" + str(subreddit) + str(counter) +".csv", index=False)
                    counter += 1
            except:
                continue
#even though the size is 400, the max is 100. They didnt update the document for pushshift api for the size.

In [ ]:
#allows for importing zip
import zipfile as zf
files = zf.ZipFile("DataReddit.zip", 'r')
files.extractall('C:/Users/Wesley Tat/STA 141B/Data')
files.close()

In [4]:
os.getcwd()

'C:\\Users\\wesle\\STA 141B'

In [ ]:
def all_csv_files(dates, subreddit, folder): #find all the csvs and makes it into a list 
    '''
    Find all the csvs and makes it into a list. It is a list within a list 
    dates: list format. It is which dates you have collected your data, essentially naming scheme. 
    subreddit: Which subreddit, naming scheme for your data. Ex: "Republican"
    folder: where are the CSVs located? Based on your working directory + folder. Str format
    '''
    csv_list = []
    for i in dates:
        csv_list.append(glob.glob(os.path.join(os.getcwd()+"\\"+str(folder), str(i) +", Subreddit-" + str(subreddit)+ "*.csv")))
    return(csv_list)

In [ ]:
df = {} #initalizing dataframe so I can have a copy
def conv_csv_to_pd(sub): 
    '''
    Purpose is to collect all the csvs we found then convert it back to dataframes and combine them with one another based on
    the subreddit we chose.
    sub: list format, should be the lsit of subreddit used. Ex: ["Republican", "Democrats","Conservatives"]
    '''
    for name in sub:
        dataset = all_csv_files(["03_11_2020-09_07_2020","09_11_2020-03_06_2021","03_06_2021-09_02_2021","09_02_2021-03_01_2022",], str(name), "Data")
        try: #in case of failure 
            for i in range(0,4):
                data_ = pd.concat(map(pd.read_csv, dataset[i]), ignore_index=True) #adds all the dataframes together 
            if name in df.keys(): #if it exists, no need and just add dataframe to df 
                df[name] = data_
            else: #making a new dataframe each time for the new subreddit
                df[name] = pd.DataFrame()
                df[name] = data_
        except:
            print("We have no data on that " + str(name) + " subreddit.")
    return(df)

In [ ]:
subreddits = ['CoronavirusCA',
 'Republican',
 'AskReddit',
 'Conservative',
 'Democrats',
 'socialism',
 'Liberal',
 'moderatepolitics',
 'UCDavis',
 'Libertarian',
 'Politics']

In [ ]:
conv_csv_to_pd(subreddits)